In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/train-ms-en.tar.gz
# !tar -zxf train-ms-en.tar.gz
# !rm train-ms-en.tar.gz

In [2]:
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon.json

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [4]:
!ls -lha train

total 890M
drwxr-xr-x  2 ubuntu ubuntu 4.0K Jun  26  2020 .
drwxr-xr-x 27 ubuntu ubuntu 4.0K Jul   3 12:35 ..
-rw-r--r--  1 ubuntu ubuntu 471M Jun  27  2020 left.txt
-rw-r--r--  1 ubuntu ubuntu 420M Jun  27  2020 right.txt


In [5]:
with open('train/left.txt') as fopen:
    left = fopen.read().split('\n')
    
with open('train/right.txt') as fopen:
    right = fopen.read().split('\n')

In [6]:
left[7106], right[7106]

('He said it was ridiculous for Apandi to state that investigations conducted by law enforcement agencies here had revealed that there was no evidence to show 1MDB money had been misappropriated.',
 'He said it was ridiculous for Apandi to state that investigations conducted by law enforcement agencies here had revealed that there was no evidence to show 1MDB money had been misappropriated.')

In [7]:
len(left), len(right)

(3712555, 3712555)

In [8]:
import malaya

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [9]:
eflomal = malaya.alignment.ms_en.eflomal()

In [10]:
%%time

eflomal.align(left[7106:7107], right[7106:7107])

CPU times: user 750 µs, sys: 36.2 ms, total: 36.9 ms
Wall time: 185 ms


{'forward': [[(0, 0),
   (1, 1),
   (2, 2),
   (3, 3),
   (4, 4),
   (5, 5),
   (6, 6),
   (7, 7),
   (8, 8),
   (9, 9),
   (10, 10),
   (11, 11),
   (12, 12),
   (13, 13),
   (14, 14),
   (15, 15),
   (16, 16),
   (17, 17),
   (18, 18),
   (19, 19),
   (20, 20),
   (21, 21),
   (22, 22),
   (23, 23),
   (24, 24),
   (25, 25),
   (26, 26),
   (27, 27),
   (28, 28),
   (29, 29),
   (30, 30)]],
 'reverse': [[(0, 0),
   (1, 1),
   (2, 2),
   (3, 3),
   (4, 4),
   (5, 5),
   (6, 6),
   (7, 7),
   (8, 8),
   (9, 9),
   (10, 10),
   (11, 11),
   (12, 12),
   (13, 13),
   (14, 14),
   (15, 15),
   (16, 16),
   (17, 17),
   (18, 18),
   (19, 19),
   (20, 20),
   (21, 21),
   (22, 22),
   (23, 23),
   (24, 24),
   (25, 25),
   (26, 26),
   (27, 27),
   (28, 28),
   (29, 29),
   (30, 30)]]}

In [11]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from collections import defaultdict
import re
import random
import json

In [12]:
with open('en-social-media-lexicon.json') as fopen:
    en_lexicon = json.load(fopen)

In [13]:
rev_rules_compound_normalizer = defaultdict(list)
for k, v in rules_compound_normalizer.items():
    rev_rules_compound_normalizer[v].append(k)

In [14]:
rev_rules_normalizer = defaultdict(list)
for k, v in rules_normalizer.items():
    rev_rules_normalizer[v].append(k)

In [15]:
rules_compound_normalizer_regex = (
    '(?:' + '|'.join(list(rev_rules_compound_normalizer.keys())) + ')'
)

In [16]:
def _replace_compound(string):
    results = re.findall(
        rules_compound_normalizer_regex, string, flags=re.IGNORECASE
    )
    for r in results:
        string = string.replace(r, random.choice(rev_rules_compound_normalizer[r.lower()]))
    return string

In [17]:
_replace_compound('sudah tentu tidak')

'of cos tidak'

In [18]:
rev_rules_normalizer

defaultdict(list,
            {'biar betul': ['seriously', 'sriusly', 'siriusly'],
             'kamu': ['lu',
              'hg',
              'kau',
              'ngko',
              'ngkau',
              'ko',
              'kamo',
              'kmu',
              'kqmu',
              'kemuh',
              'qmu',
              'kamyuhhh',
              'kam',
              'luhhh',
              'amu',
              'amohhh',
              'luh'],
             'pun': ['pon', 'pn'],
             'sahaja lah': ['jela', 'jelah'],
             'tiada': ['tkde', 'tkda', 'takda', 'gda', 'gada'],
             'pergi': ['pegih', 'pegi', 'g', 'go', 'leave', 'p', 'pg'],
             'dahulu': ['dulu', 'dlu', 'doeloe'],
             'sah': ['confirm', 'kompom'],
             'disahkan': ['confirmed'],
             'secara asasnya': ['basically'],
             'doktor': ['dokter'],
             'celaka': ['hareeii',
              'hareii',
              'celako',
              'hawau',


In [19]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_


def replace_shortword(word, rules = rev_rules_normalizer):
    left, right, word_ = strip_punct(word)
    word_ = word_[len(left):]
    lower_word_ = word_.lower()
    if lower_word_ in rules:
        word_ = case_of(word_)(random.choice(rules[lower_word_]))
        word_ = f'{left}{word_}{right}'
        return word_
    else:
        return word
    
strip_punct('counters,')

('', ',', 'counters')

In [20]:
string = 'saya sangat cantik"'
[replace_shortword(word) for word in string.split()]

['i', 'banged', 'catiiiiik"']

In [21]:
[replace_shortword(word) for word in left[1].split()]

['"Pembangkang',
 'hnya',
 'brani',
 'cakap',
 'big,',
 'seolah-olah',
 'indah',
 'rupa',
 'dripda',
 'brita,"',
 'ktanya.']

In [22]:
[replace_shortword(word, rules = en_lexicon) for word in 'do you like chicken'.split()]

['do', 'yew', 'liek', 'chicken']

In [23]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 2, max_replace = 7):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
            continue
        elif splitted_left[l].isupper() or splitted_right[r].isupper():
            continue
        elif splitted_left[l] == splitted_right[r]:
            continue
        elif splitted_right[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
            continue
        else:
            selected_alignment.append((l, r))
    
    count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
    
    selected = random.sample(selected_alignment, count_replace)
    for s in selected:
        splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])
    
    return ' '.join(splitted_left)

In [24]:
# random replace alignment
# random replace compound
# random replace word

In [25]:
random.random()

0.9813474370566361

In [26]:
def augment(left, right, p_replace_alignment = 0.4, p_replace_shortword = 0.7, p_replace_en_shortword = 0.3):
    if random.random() > p_replace_alignment:
        alignment = eflomal.align([left], [right])['forward'][0]
        left = random_replace_alignment(left, right, alignment)
        left = ' '.join([replace_shortword(word, rules = en_lexicon) if random.random() > p_replace_en_shortword else word for word in left.split()])
        
    
    left = _replace_compound(left)
    left = ' '.join([replace_shortword(word) if random.random() > p_replace_shortword else word for word in left.split()])
    return left

In [27]:
fast_text = malaya.language_detection.fasttext()

In [28]:
fast_text.predict([left[7106], right[7106], 'saya suka'])

['eng', 'eng', 'malay']

In [29]:
from tqdm import tqdm

def loop(rows):
    rows, _ = rows
    new_left, new_right = [], []
    for i in tqdm(range(len(rows))):
        left, right = rows[i][0], rows[i][1]
        if len(left.split()) > 100 or len(right.split()) > 100:
            continue
        langs = fast_text.predict([left, right])
        if langs[0] == 'eng':
            continue
        if langs[1] == 'malay':
            continue
        try:
            new_left_ = augment(left, right)
            if new_left_ != left:
                new_left.append(new_left_)
                new_right.append(right)
        except Exception as e:
            pass
    return [[new_left, new_right]]

In [30]:
loop((list(zip(left[:10], right[:10])),0))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.67it/s]


[[['"opposition hny berani shayy besar, seolah-olah indah rupa daripada berita," he.',
   'Salah satu fakta yang paling mungkin adl orang Arab berniaga untuk hamba abdi di Afrika, meningkatkan penyebaran dan dominasi budaya Arab.',
   'Macam Wut abt JoonPyo?',
   'Bogazkoy, Amasya Bogazkoy, Amasya merupakan sebuah region yang located i Turkey.',
   'Beliau dianggap sebagai ahli falsafah yanckh paling agung pernah dilahirkan in history Islam.',
   'And bacakanlah (wahai Muhammad) di dalam Scripture (Abraham) ini perihal Nabi Ibrahim; He adalah ws sorang of amat truth, lagi mnjadi Nabi.',
   'Tersedia scra nasional melalui dua layanan satelit di negara thu Star Choice, pada saluran 276 dan Bell TV pada saluran 828.',
   'Pelapis bru pun dah jadi lama also including Kaa Teh.',
   'Pada masa ini, jemaah menteri Dr Mahathir hanya mempunyai 13 anggota sedangkan ada beberapa portfolio penting termasuk Menteri Luar yg masih kosong selain tiada timbalan menteri dinamakn.'],
  ['"The opposition 

In [31]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [32]:
import mp

In [36]:
r = mp.multiprocessing(list(zip(left[1100000:1200100], right[1100000:1200100])), loop, cores = 1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 100100/100100 [3:21:32<00:00,  8.28it/s]


In [38]:
len(r[0])

2

In [39]:
r[0][0][:10]

['Sebelum iniehh, Tabung Kanak-Kanak United Nations Bersatu or UNICEF dalam report yg bertajuk "Children Without: A study of urban chill poverty and deprivation i low-cost flats in Kuala Lumpur" menunujukkan 99.7 peratus children Malaysia of livin di kawasan rumah kos rendah didapati kurang zat nutrients.',
 '"Kita pinginnn kedamaian, inilah diajar dalam al-Quran, kedamaian merupakan kunci kbahagiaan dan hny akan dapet diterjemahkan melalui kesatuan, apabila hati kita sling meraikan e unnerstand, gmn kta interact dengan seluru masyarakat," ktny.',
 'Teori biogenic ini mula-mula diperkenalkan oleh Georg Agricola i tahun 1556, then oleh Mikhail Lomonosov pada abad ke-18.',
 '"Negara kita berbilang kaum namun kejayaan kaum tidak sama ngan kaum Melayu perlu diberikan peluang untuk terus berjaya.',
 'Dan (amatlah ngerinya) kalau engkau melihat (wahai orang yang memandang), ktika malaikat mengambil nyawa org2 kafir by memukul muka & belakang mereka (sambil berkata): "Rasalah kamu azab seksa 

In [40]:
r[0][1][:10]

['Previously, the United Nations Children\'s Fund or UNICEF in a report entitled "Children Without: A study of urban child poverty and deprivation in low-cost flats in Kuala Lumpur" targeted 99.7 percent of Malaysian children living in the area low-cost homes are found to be low in nutrients.',
 '"We want peace, this is taught in the Quran, peace is the key to happiness and can only be translated through unity, when our hearts celebrate and understand, how we interact with the whole community," he said.',
 'This biogenic theory was first introduced by Georg Agricola in 1556, then by Mikhail Lomonosov in the 18th century.',
 '"Our country races but not the same success with the Malays should be given a chance to succeed.',
 "Were you to see when the angels take away the faithless, striking their faces and their backs, [saying], 'Taste the punishment of the burning.",
 "That's the wavelength of light it absorbs best.",
 'For the heavyweights, Maybank gained two sen to RM9.',
 '"Therefore

In [41]:
import json

with open('augmented-ms-en-v2.json', 'w') as fopen:
    json.dump({'en': r[0][1], 'ms': r[0][0]}, fopen)

In [42]:
with open('augmented-ms-en-v2.json') as fopen:
    data = json.load(fopen)